In [1]:
import pandas as pd
from utils import *
import itertools

## Affected Importers Label

A country is labelled as affected importer if:
 * One of its exporter partners has "lost" its condition as exporter
 * The importer either reduces its imports for the commodity by an amount of 20%; OR
 * The importer pays a higher price than the previous year, in excess of the price change for the commodity

Retrieve BACI data to compute average prices

In [3]:
avg_prices = pd.DataFrame()
avg_prices_per_importer = pd.DataFrame()
atlas_countries = pd.concat([pd.read_stata(f"{data_paths['atlas']}/hs12_country_country_product_year_4_2012_2016.dta"),
                pd.read_stata(f"{data_paths['atlas']}/hs12_country_country_product_year_4_2017_2021.dta"),
                pd.read_stata(f"{data_paths['atlas']}/hs12_country_country_product_year_4_2022.dta")]).country_id.unique()
all_countries = set(atlas_countries)

for year in range(2012, 2023):
    print(year)
    baci = pd.read_csv(f"{data_paths['baci']}/BACI_HS12_Y{year}_V202501.csv", dtype={"product_id": str})
    baci["product_code"] = baci.product_id.str[:2] ## Reduce product code to 2 digits
    baci.dropna(inplace=True) # Some entries have NANs for qty. We'll dismiss this faulty entries.
    # Calculate the average price each importer was paying for each product (across all partners)
    avg_prices_per_importer_year = baci.groupby(["importer", "product_code"])[["export_value", "qty"]].sum().reset_index()
    avg_prices_per_importer_year["price"] = avg_prices_per_importer_year.export_value / avg_prices_per_importer_year.qty
    avg_prices_per_importer_year.loc[:, ["year"]] = year
    avg_prices_per_importer = pd.concat([avg_prices_per_importer, avg_prices_per_importer_year])
    # Calculate average price per commodity-year
    avg_prices_year = baci.groupby("product_code")[["export_value", "qty"]].sum().reset_index()
    avg_prices_year["price"] = avg_prices_year.export_value / avg_prices_year.qty
    avg_prices_year["year"] = year
    avg_prices = pd.concat([avg_prices, avg_prices_year])
    # Update all seen countries
    all_countries.update(baci.exporter)
    all_countries.update(baci.importer)
    del baci # to recover some memory

2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


Get the report of potentially affected importers by 'lost exporters'

In [4]:
# Initial report of potentially affected importers from lost exporters
drop_exporters_impact = pd.read_csv(f"{data_paths['graphs_data']}/df_impact_drop_exporters.csv")
drop_exporters_impact.rename(columns={"pid": "product_code"}, inplace=True)
drop_exporters_impact.head(3)

,affected_importer,year_from_value,year_to_value,year_from_n_exporters,year_to_n_exporters,year_from,year_to,product_code,exporter_id,drop_pct
0,276,2.339764e+09,3.153808e+09,517.0,525.0,2012,2013,86,4,-0.584686
1,528,7.063663e+08,4.552369e+08,298.0,309.0,2012,2013,86,4,-0.584686
2,586,1.444143e+08,6.381490e+07,58.0,54.0,2012,2013,86,4,-0.584686


In [5]:
# Calculate drop in imports from potentially affected importers
drop_exporters_impact["importer_drop"] = (drop_exporters_impact["year_to_value"] / drop_exporters_impact["year_from_value"]) - 1
drop_exporters_impact.head(3)

,affected_importer,year_from_value,year_to_value,year_from_n_exporters,year_to_n_exporters,year_from,year_to,product_code,exporter_id,drop_pct,importer_drop
0,276,2.339764e+09,3.153808e+09,517.0,525.0,2012,2013,86,4,-0.584686,0.347917
1,528,7.063663e+08,4.552369e+08,298.0,309.0,2012,2013,86,4,-0.584686,-0.355523
2,586,1.444143e+08,6.381490e+07,58.0,54.0,2012,2013,86,4,-0.584686,-0.558112


We need to find the products that 'affected importer' was importing from 'lost exporters' at 'year from', then add the avg. price of the commodity, then do the same for 'year to', and calculate the ratios.

In [6]:
# Get Avg. Price Product (APP) for year from
drop_exporters_impact = drop_exporters_impact.merge(avg_prices[["product_code", "year", "price"]], left_on=["product_code", "year_from"], \
                                                    right_on=["product_code", "year"], how="left")
drop_exporters_impact.drop("year", axis=1, inplace=True)
drop_exporters_impact.rename(columns={"price": "APP_year_from"}, inplace=True)
# Get Avg. Price Product (APP) for year to
drop_exporters_impact = drop_exporters_impact.merge(avg_prices[["product_code", "year", "price"]], left_on=["product_code", "year_to"], \
                                                    right_on=["product_code", "year"], how="left")
drop_exporters_impact.drop("year", axis=1, inplace=True)
drop_exporters_impact.rename(columns={"price": "APP_year_to"}, inplace=True)
# APP Change
drop_exporters_impact.loc[:, ["APP_Change"]] = (drop_exporters_impact.APP_year_to / drop_exporters_impact.APP_year_from) - 1

In [7]:
drop_exporters_impact.head(3)

,affected_importer,year_from_value,year_to_value,year_from_n_exporters,year_to_n_exporters,year_from,year_to,product_code,exporter_id,drop_pct,importer_drop,APP_year_from,APP_year_to,APP_Change
0,276,2.339764e+09,3.153808e+09,517.0,525.0,2012,2013,86,4,-0.584686,0.347917,4.857616,4.935438,0.016021
1,528,7.063663e+08,4.552369e+08,298.0,309.0,2012,2013,86,4,-0.584686,-0.355523,4.857616,4.935438,0.016021
2,586,1.444143e+08,6.381490e+07,58.0,54.0,2012,2013,86,4,-0.584686,-0.558112,4.857616,4.935438,0.016021


We need to find the products that 'affected importer' was importing from 'lost exporters' at 'year from', then add the avg. price paid by importer and avg. price of the commodity, then do the same for 'year to', and calculate the ratios.

In [8]:
# Get Avg. Price Product (APP) for year from
drop_exporters_impact = drop_exporters_impact.merge(avg_prices_per_importer[["importer", "product_code", "year", "price"]], \
                                                    left_on=["affected_importer", "product_code", "year_from"], \
                                                    right_on=["importer", "product_code", "year"], how="left")
drop_exporters_impact.drop(["importer", "year"], axis=1, inplace=True)
drop_exporters_impact.rename(columns={"price": "CPP_year_from"}, inplace=True)
# Get Avg. Price Product (APP) for year to
drop_exporters_impact = drop_exporters_impact.merge(avg_prices_per_importer[["importer", "product_code", "year", "price"]], \
                                                    left_on=["affected_importer", "product_code", "year_to"], \
                                                    right_on=["importer", "product_code", "year"], how="left")
drop_exporters_impact.drop(["importer", "year"], axis=1, inplace=True)
drop_exporters_impact.rename(columns={"price": "CPP_year_to"}, inplace=True)
# APP Change
drop_exporters_impact.loc[:, ["expected_change"]] = drop_exporters_impact.CPP_year_from * (1+drop_exporters_impact.APP_Change)

In [9]:
drop_exporters_impact["net_change"] = (drop_exporters_impact.CPP_year_to / drop_exporters_impact.expected_change) - 1

In [10]:
drop_exporters_impact[(drop_exporters_impact.product_code == "01") & (drop_exporters_impact.year_from == 2012) & (drop_exporters_impact.affected_importer == 8)][["affected_importer", "year_from", "year_to", "APP_year_from", "APP_year_to", "CPP_year_from", "CPP_year_to", "expected_change", "net_change"]]

,affected_importer,year_from,year_to,APP_year_from,APP_year_to,CPP_year_from,CPP_year_to,expected_change,net_change
457,8,2012,2013,3.02615,3.099866,1.897458,1.919112,1.943678,-0.012639


Apply conditions for affected label

In [11]:
affected_importers = drop_exporters_impact[(drop_exporters_impact.importer_drop <= -0.2) | (drop_exporters_impact.net_change >= 0.2)]

Make sure all labels are present

In [13]:
reporters = pd.read_csv(f"{data_paths['uncomtrade']}/reporters.csv")
reporters.rename(columns={"reporterCode": "country_id"}, inplace=True)
all_countries.update(reporters.country_id) # Add countries from reporters (UN Comtrade)

# Generate all combinations
all_combinations = pd.DataFrame(itertools.product(all_countries, list(range(2012,2022)), \
                                                  list([f'{x:02d}' for x in range(1, 99) if x not in [77, 98]])), columns=['country_id', 'year', 'product_code'])

In [14]:
affected_importers.head()

,affected_importer,year_from_value,year_to_value,year_from_n_exporters,year_to_n_exporters,year_from,year_to,product_code,exporter_id,drop_pct,importer_drop,APP_year_from,APP_year_to,APP_Change,CPP_year_from,CPP_year_to,expected_change,net_change
0,276,2.339764e+09,3.153808e+09,517.0,525.0,2012,2013,86,4,-0.584686,0.347917,4.857616,4.935438,0.016021,4.960357,7.298090,5.039825,0.448084
1,528,7.063663e+08,4.552369e+08,298.0,309.0,2012,2013,86,4,-0.584686,-0.355523,4.857616,4.935438,0.016021,4.841990,1.864074,4.919562,-0.621089
2,586,1.444143e+08,6.381490e+07,58.0,54.0,2012,2013,86,4,-0.584686,-0.558112,4.857616,4.935438,0.016021,18.398550,8.800903,18.693305,-0.529195
6,528,7.275954e+07,1.475283e+08,145.0,159.0,2012,2013,93,4,-0.736298,1.027615,31.745987,33.384077,0.051600,40.542424,65.215549,42.634410,0.529646
7,276,3.664020e+07,3.889862e+07,143.0,141.0,2012,2013,14,8,-0.238721,0.061638,0.344513,0.338611,-0.017133,0.545503,0.813757,0.536157,0.517759


In [15]:
affected_importers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6409 entries, 0 to 26664
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   affected_importer      6409 non-null   int64  
 1   year_from_value        6409 non-null   float64
 2   year_to_value          6409 non-null   float64
 3   year_from_n_exporters  6409 non-null   float64
 4   year_to_n_exporters    6409 non-null   float64
 5   year_from              6409 non-null   int64  
 6   year_to                6409 non-null   int64  
 7   product_code           6409 non-null   object 
 8   exporter_id            6409 non-null   int64  
 9   drop_pct               6409 non-null   float64
 10  importer_drop          6409 non-null   float64
 11  APP_year_from          6028 non-null   float64
 12  APP_year_to            6028 non-null   float64
 13  APP_Change             6028 non-null   float64
 14  CPP_year_from          5814 non-null   float64
 15  CPP_year

In [16]:
affected_importers.fillna(0, inplace=True)

C:\Users\ginof\AppData\Local\Temp\ipykernel_27104\3713369678.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  affected_importers.fillna(0, inplace=True)


In [17]:
affected_importers["label"] = "affected_importer"
affected_importers.rename(columns={"affected_importer": "country_id", "year_from": "year"}, inplace=True)
affected_importers = all_combinations.merge(affected_importers[["country_id", "year", "product_code", "label"]], on=["year", "product_code", "country_id"], how="left")
affected_importers.fillna("not_affected", inplace=True)

C:\Users\ginof\AppData\Local\Temp\ipykernel_27104\373663775.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  affected_importers["label"] = "affected_importer"
C:\Users\ginof\AppData\Local\Temp\ipykernel_27104\373663775.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  affected_importers.rename(columns={"affected_importer": "country_id", "year_from": "year"}, inplace=True)


In [18]:
affected_importers.head(3)

,country_id,year,product_code,label
0,512,2012,01,not_affected
1,512,2012,02,not_affected
2,512,2012,03,not_affected


In [19]:
affected_importers.groupby(["year", "product_code", "label"]).size()

year  product_code  label            
2012  01            affected_importer     10
                    not_affected         266
      02            affected_importer      3
                    not_affected         273
      03            affected_importer      2
                                        ... 
2021  95            affected_importer      2
                    not_affected         274
      96            not_affected         276
      97            affected_importer     24
                    not_affected         256
Length: 1581, dtype: int64

In [20]:
#affected_importers.sort_values(["year", "product_code", "country_id"])[["year", "product_code", "country_id", "label"]].to_csv("labels-affected_importers.csv", index=False)